In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
def evaluate_fairness(y_true, y_pred, sensitive_features):
        """
        Evaluates fairness of the final majority vote classifier over T_inner hypotheses
        on the test set.
        #NOTE: defined in the meta_algo file, but we chose:
        a0 := African-American (COMPAS), Female (Adult)
        a1 := Caucasian (COMPAS), Male (Adult)

        :return: list. subgroups in sensitive_features.
        :return: dict. recidivism_pct for each group.
        """
        groups = np.unique(sensitive_features.values)
        pos_count = {}
        dp_pct = {}
        eo_y0_pct = {}
        eo_y1_pct = {}
        
        for index, group in enumerate(groups):
            # Demographic Parity
            indices = {}
            indices[group] = sensitive_features.index[sensitive_features == group]
            dp_pct[group] = sum(y_pred[indices[group]])/len(indices[group])

            # Equalized Odds
            y1_indices = {}
            y0_indices = {}
            y1_indices[group] = sensitive_features.index[(sensitive_features == group) & (y_true == 1)]
            y0_indices[group] = sensitive_features.index[(sensitive_features == group) & (y_true == 0)]
            eo_y0_pct[group] = sum(y_pred[y0_indices[group]])/len(y0_indices[group])   
            eo_y1_pct[group] = sum(y_pred[y1_indices[group]])/len(y1_indices[group])
        
        gaps = {}
        group_metrics = {} # a dictionary of dictionaries

        gaps['dp'] = abs(dp_pct[groups[0]] - dp_pct[groups[1]])
        gaps['eo_y0'] = abs(eo_y0_pct[groups[0]] - eo_y0_pct[groups[1]])
        gaps['eo_y1'] = abs(eo_y1_pct[groups[0]] - eo_y1_pct[groups[1]])
        group_metrics['dp'] = dp_pct
        group_metrics['eo_y0'] = eo_y0_pct
        group_metrics['eo_y1'] = eo_y1_pct
        
        return groups, group_metrics, gaps

## Compas Dataset

In [5]:
compas_train = pd.read_csv('./data/processed/compas/compas_train1_X.csv')
compas_test = pd.read_csv('./data/processed/compas/compas_test1_X.csv')

In [4]:
compas_train.head(5)

,sex,race,age_cat=25 to 45,age_cat=Greater than 45,age_cat=Less than 25,priors_count=0,priors_count=1 to 3,priors_count=More than 3,c_charge_degree=F,c_charge_degree=M,two_year_recid
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
4,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0


In [5]:
y_train = compas_train.pop('two_year_recid') 
y_test = compas_test.pop('two_year_recid')
sensitive_features_train = compas_train['race']
sensitive_features_test = compas_test['race']
X_train = compas_train
X_test = compas_test

In [6]:
sensitive_features_test.head(5)

0    0.0
1    0.0
2    1.0
3    0.0
4    0.0
Name: race, dtype: float64

In [7]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

groups, group_metrics, gaps = evaluate_fairness(y_test, y_pred, sensitive_features_test)

print("Logistic Regression Accuracy: " + str(accuracy_score(y_pred, y_test)))

# Demographic Parity
for group in groups:
      print("P[h(X) = 1 | {}] = {}".format(group, group_metrics['dp'][group]))
print("Delta_dp = {}".format(gaps['dp']))

# Equalized Odds
for group in groups:
    print("P[h(X) = 1 | A = {}, Y = 1] = {}".format(group, group_metrics['eo_y1'][group]))
    print("P[h(X) = 1 | A = {}, Y = 0] = {}".format(group, group_metrics['eo_y0'][group]))
print("Delta_eo1 = {}".format(gaps['eo_y1']))
print("Delta_eo0 = {}".format(gaps['eo_y0']))

Logistic Regression Accuracy: 0.6590909090909091
P[h(X) = 1 | 0.0] = 0.5123456790123457
P[h(X) = 1 | 1.0] = 0.19607843137254902
Delta_dp = 0.31626724763979674
P[h(X) = 1 | A = 0.0, Y = 1] = 0.6612021857923497
P[h(X) = 1 | A = 0.0, Y = 0] = 0.3191489361702128
P[h(X) = 1 | A = 1.0, Y = 1] = 0.2962962962962963
P[h(X) = 1 | A = 1.0, Y = 0] = 0.13008130081300814
Delta_eo1 = 0.3649058894960534
Delta_eo0 = 0.18906763535720464


In [8]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

groups, group_metrics, gaps = evaluate_fairness(y_test, y_pred, sensitive_features_test)

print("Random Forest Accuracy: " + str(accuracy_score(y_pred, y_test)))

# Demographic Parity
for group in groups:
      print("P[h(X) = 1 | {}] = {}".format(group, group_metrics['dp'][group]))
print("Delta_dp = {}".format(gaps['dp']))

# Equalized Odds
for group in groups:
    print("P[h(X) = 1 | A = {}, Y = 1] = {}".format(group, group_metrics['eo_y1'][group]))
    print("P[h(X) = 1 | A = {}, Y = 0] = {}".format(group, group_metrics['eo_y0'][group]))
print("Delta_eo1 = {}".format(gaps['eo_y1']))
print("Delta_eo0 = {}".format(gaps['eo_y0']))

Random Forest Accuracy: 0.6458333333333334
P[h(X) = 1 | 0.0] = 0.5030864197530864
P[h(X) = 1 | 1.0] = 0.22549019607843138
Delta_dp = 0.2775962236746551
P[h(X) = 1 | A = 0.0, Y = 1] = 0.6284153005464481
P[h(X) = 1 | A = 0.0, Y = 0] = 0.3404255319148936
P[h(X) = 1 | A = 1.0, Y = 1] = 0.345679012345679
P[h(X) = 1 | A = 1.0, Y = 0] = 0.14634146341463414
Delta_eo1 = 0.2827362882007691
Delta_eo0 = 0.19408406850025947


## Adult Dataset

In [9]:
adult_X = pd.read_csv('./data/adult_X.csv')
adult_y = pd.read_csv('./data/adult_y.csv')

adult_X.head(5)

X_train, X_test, y_train, y_test = train_test_split(adult_X, adult_y, test_size=0.2, random_state=42)
y_test = y_test.reset_index(drop=True)
y_test = y_test['income']
sensitive_features_train = X_train['sex']
sensitive_features_test = X_test['sex']

sensitive_features_train[sensitive_features_train < 0] = 0
sensitive_features_train[sensitive_features_train > 0] = 1
sensitive_features_train = sensitive_features_train.reset_index(drop=True)

sensitive_features_test[sensitive_features_test < 0] = 0
sensitive_features_test[sensitive_features_test > 0] = 1
sensitive_features_test = sensitive_features_test.reset_index(drop=True)

/Users/sam/anaconda3/envs/fairness_checking/lib/python3.7/site-packages/pandas/core/generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [11]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print("Logistic Regression Accuracy: " + str(accuracy_score(y_pred, y_test)))
groups, group_metrics, gaps = evaluate_fairness(y_test, y_pred, sensitive_features_test)

# Demographic Parity
for group in groups:
      print("P[h(X) = 1 | {}] = {}".format(group, group_metrics['dp'][group]))
print("Delta_dp = {}".format(gaps['dp']))

# Equalized Odds
for group in groups:
    print("P[h(X) = 1 | A = {}, Y = 1] = {}".format(group, group_metrics['eo_y1'][group]))
    print("P[h(X) = 1 | A = {}, Y = 0] = {}".format(group, group_metrics['eo_y0'][group]))
print("Delta_eo1 = {}".format(gaps['eo_y1']))
print("Delta_eo0 = {}".format(gaps['eo_y0']))

Logistic Regression Accuracy: 0.8267326732673267
P[h(X) = 1 | 0.0] = 0.6566666666666666
P[h(X) = 1 | 1.0] = 0.18269230769230768
Delta_dp = 0.4739743589743589
P[h(X) = 1 | A = 0.0, Y = 1] = 0.8839779005524862
P[h(X) = 1 | A = 0.0, Y = 0] = 0.31092436974789917
P[h(X) = 1 | A = 1.0, Y = 1] = 0.6521739130434783
P[h(X) = 1 | A = 1.0, Y = 0] = 0.04938271604938271
Delta_eo1 = 0.23180398750900788
Delta_eo0 = 0.26154165369851645


/Users/sam/anaconda3/envs/fairness_checking/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

groups, group_metrics, gaps = evaluate_fairness(y_test, y_pred, sensitive_features_test)

print("Random Forest Accuracy: " + str(accuracy_score(y_pred, y_test)))

# Demographic Parity
for group in groups:
      print("P[h(X) = 1 | {}] = {}".format(group, group_metrics['dp'][group]))
print("Delta_dp = {}".format(gaps['dp']))

# Equalized Odds
for group in groups:
    print("P[h(X) = 1 | A = {}, Y = 1] = {}".format(group, group_metrics['eo_y1'][group]))
    print("P[h(X) = 1 | A = {}, Y = 0] = {}".format(group, group_metrics['eo_y0'][group]))
print("Delta_eo1 = {}".format(gaps['eo_y1']))
print("Delta_eo0 = {}".format(gaps['eo_y0']))

/Users/sam/anaconda3/envs/fairness_checking/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Random Forest Accuracy: 0.8316831683168316
P[h(X) = 1 | 0.0] = 0.6366666666666667
P[h(X) = 1 | 1.0] = 0.18269230769230768
Delta_dp = 0.453974358974359
P[h(X) = 1 | A = 0.0, Y = 1] = 0.8784530386740331
P[h(X) = 1 | A = 0.0, Y = 0] = 0.2689075630252101
P[h(X) = 1 | A = 1.0, Y = 1] = 0.6086956521739131
P[h(X) = 1 | A = 1.0, Y = 0] = 0.06172839506172839
Delta_eo1 = 0.26975738650012004
Delta_eo0 = 0.20717916796348168


## Law School Dataset

In [13]:
lawschool_X = pd.read_csv('./data/lawschool_X.csv')
lawschool_y = pd.read_csv('./data/lawschool_y.csv')

X_train, X_test, y_train, y_test = train_test_split(lawschool_X, lawschool_y, test_size=0.2, random_state=42)
y_test = y_test.reset_index(drop=True)
y_test = y_test['bar1']
sensitive_features_train = X_train['race7']
sensitive_features_test = X_test['race7']

sensitive_features_train[sensitive_features_train < 0] = 0
sensitive_features_train[sensitive_features_train > 0] = 1
sensitive_features_train = sensitive_features_train.reset_index(drop=True)

sensitive_features_test[sensitive_features_test < 0] = 0
sensitive_features_test[sensitive_features_test > 0] = 1
sensitive_features_test = sensitive_features_test.reset_index(drop=True)

/Users/sam/anaconda3/envs/fairness_checking/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/sam/anaconda3/envs/fairness_checking/lib/python3.7/site-packages/pandas/core/generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/Users/sam/anaconda3/envs/fairness_checking/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.or

In [15]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print("Logistic Regression Accuracy: " + str(accuracy_score(y_pred, y_test)))
groups, group_metrics, gaps = evaluate_fairness(y_test, y_pred, sensitive_features_test)

# Demographic Parity
for group in groups:
      print("P[h(X) = 1 | {}] = {}".format(group, group_metrics['dp'][group]))
print("Delta_dp = {}".format(gaps['dp']))

# Equalized Odds
for group in groups:
    print("P[h(X) = 1 | A = {}, Y = 1] = {}".format(group, group_metrics['eo_y1'][group]))
    print("P[h(X) = 1 | A = {}, Y = 0] = {}".format(group, group_metrics['eo_y0'][group]))
print("Delta_eo1 = {}".format(gaps['eo_y1']))
print("Delta_eo0 = {}".format(gaps['eo_y0']))

Logistic Regression Accuracy: 0.7945205479452054
P[h(X) = 1 | 0] = 0.2403846153846154
P[h(X) = 1 | 1] = 0.5862068965517241
Delta_dp = 0.34582228116710867
P[h(X) = 1 | A = 0, Y = 1] = 0.5294117647058824
P[h(X) = 1 | A = 0, Y = 0] = 0.1
P[h(X) = 1 | A = 1, Y = 1] = 0.806060606060606
P[h(X) = 1 | A = 1, Y = 0] = 0.20833333333333334
Delta_eo1 = 0.2766488413547237
Delta_eo0 = 0.10833333333333334


/Users/sam/anaconda3/envs/fairness_checking/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sam/anaconda3/envs/fairness_checking/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [16]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

groups, group_metrics, gaps = evaluate_fairness(y_test, y_pred, sensitive_features_test)

print("Random Forest Accuracy: " + str(accuracy_score(y_pred, y_test)))

# Demographic Parity
for group in groups:
      print("P[h(X) = 1 | {}] = {}".format(group, group_metrics['dp'][group]))
print("Delta_dp = {}".format(gaps['dp']))

# Equalized Odds
for group in groups:
    print("P[h(X) = 1 | A = {}, Y = 1] = {}".format(group, group_metrics['eo_y1'][group]))
    print("P[h(X) = 1 | A = {}, Y = 0] = {}".format(group, group_metrics['eo_y0'][group]))
print("Delta_eo1 = {}".format(gaps['eo_y1']))
print("Delta_eo0 = {}".format(gaps['eo_y0']))

Random Forest Accuracy: 0.7808219178082192
P[h(X) = 1 | 0] = 0.20192307692307693
P[h(X) = 1 | 1] = 0.6053639846743295
Delta_dp = 0.4034409077512526
P[h(X) = 1 | A = 0, Y = 1] = 0.5
P[h(X) = 1 | A = 0, Y = 0] = 0.05714285714285714
P[h(X) = 1 | A = 1, Y = 1] = 0.8
P[h(X) = 1 | A = 1, Y = 0] = 0.2708333333333333
Delta_eo1 = 0.30000000000000004
Delta_eo0 = 0.21369047619047618


/Users/sam/anaconda3/envs/fairness_checking/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


## Communities Dataset

In [17]:
communities_X = pd.read_csv('./data/communities_X.csv')
communities_y = pd.read_csv('./data/communities_y.csv')

X_train, X_test, y_train, y_test = train_test_split(communities_X, communities_y, test_size=0.2, random_state=42)
y_test = y_test.reset_index(drop=True)
y_test = y_test['ViolentCrimesPerPop']
sensitive_features_train = X_train['majority_white']
sensitive_features_test = X_test['majority_white']

sensitive_features_train[sensitive_features_train < 0] = 0
sensitive_features_train[sensitive_features_train > 0] = 1
sensitive_features_train = sensitive_features_train.reset_index(drop=True)

sensitive_features_test[sensitive_features_test < 0] = 0
sensitive_features_test[sensitive_features_test > 0] = 1
sensitive_features_test = sensitive_features_test.reset_index(drop=True)

/Users/sam/anaconda3/envs/fairness_checking/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/sam/anaconda3/envs/fairness_checking/lib/python3.7/site-packages/pandas/core/generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/Users/sam/anaconda3/envs/fairness_checking/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.or

In [18]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print("Logistic Regression Accuracy: " + str(accuracy_score(y_pred, y_test)))
groups, group_metrics, gaps = evaluate_fairness(y_test, y_pred, sensitive_features_test)

# Demographic Parity
for group in groups:
      print("P[h(X) = 1 | {}] = {}".format(group, group_metrics['dp'][group]))
print("Delta_dp = {}".format(gaps['dp']))

# Equalized Odds
for group in groups:
    print("P[h(X) = 1 | A = {}, Y = 1] = {}".format(group, group_metrics['eo_y1'][group]))
    print("P[h(X) = 1 | A = {}, Y = 0] = {}".format(group, group_metrics['eo_y0'][group]))
print("Delta_eo1 = {}".format(gaps['eo_y1']))
print("Delta_eo0 = {}".format(gaps['eo_y0']))

Logistic Regression Accuracy: 0.8696741854636592
P[h(X) = 1 | 0] = 0.6699029126213593
P[h(X) = 1 | 1] = 0.060810810810810814
Delta_dp = 0.6090921018105484
P[h(X) = 1 | A = 0, Y = 1] = 0.8428571428571429
P[h(X) = 1 | A = 0, Y = 0] = 0.30303030303030304
P[h(X) = 1 | A = 1, Y = 1] = 0.3333333333333333
P[h(X) = 1 | A = 1, Y = 0] = 0.019455252918287938
Delta_eo1 = 0.5095238095238095
Delta_eo0 = 0.2835750501120151


/Users/sam/anaconda3/envs/fairness_checking/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/sam/anaconda3/envs/fairness_checking/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [19]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

groups, group_metrics, gaps = evaluate_fairness(y_test, y_pred, sensitive_features_test)

print("Random Forest Accuracy: " + str(accuracy_score(y_pred, y_test)))

# Demographic Parity
for group in groups:
      print("P[h(X) = 1 | {}] = {}".format(group, group_metrics['dp'][group]))
print("Delta_dp = {}".format(gaps['dp']))

# Equalized Odds
for group in groups:
    print("P[h(X) = 1 | A = {}, Y = 1] = {}".format(group, group_metrics['eo_y1'][group]))
    print("P[h(X) = 1 | A = {}, Y = 0] = {}".format(group, group_metrics['eo_y0'][group]))
print("Delta_eo1 = {}".format(gaps['eo_y1']))
print("Delta_eo0 = {}".format(gaps['eo_y0']))

/Users/sam/anaconda3/envs/fairness_checking/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Random Forest Accuracy: 0.8596491228070176
P[h(X) = 1 | 0] = 0.6601941747572816
P[h(X) = 1 | 1] = 0.057432432432432436
Delta_dp = 0.6027617423248491
P[h(X) = 1 | A = 0, Y = 1] = 0.8
P[h(X) = 1 | A = 0, Y = 0] = 0.36363636363636365
P[h(X) = 1 | A = 1, Y = 1] = 0.3333333333333333
P[h(X) = 1 | A = 1, Y = 0] = 0.01556420233463035
Delta_eo1 = 0.46666666666666673
Delta_eo0 = 0.3480721613017333


In [21]:
communities_X.head[:10]

TypeError: 'method' object is not subscriptable